In [1]:
import os
import random
import keras
import numpy as np
import matplotlib.pyplot as plt

In [18]:
classes = ["hI", "ka", "rI", "sa", "tI"]

def getCVData(folder):
	data = []

	for k in range(5):
		for i in os.listdir(f"./CV_Data/{classes[k]}/{folder}"):

			with open(f"./CV_Data/{classes[k]}/{folder}/{i}") as f:
				file = f.readlines()

				vec = []
				for line in file:
					vec.append([float(x) for x in line.rstrip().split()])

				label = [0, 0, 0, 0, 0]
				label[k] = 1

				letter = [label, vec]
				data.append(letter)
	
	random.shuffle(data)
	return data

In [19]:
train = getCVData("Train")
test = getCVData("Test")

In [36]:
def split(data):
	xdata = []
	ydata = []
	for i in range(len(data)):
		xdata.append(data[i][1])
		ydata.append(data[i][0])

	return np.array(xdata), np.array(ydata)

In [37]:
xtrain, ytrain = split(train)
xtest, ytest = split(test)

C:\Users\ANIMO\AppData\Local\Temp/ipykernel_5244/3007137227.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(xdata), np.array(ydata)


In [29]:
len(train)

1439

In [38]:
model = keras.Sequential()
model.add(keras.layers.LSTM(5, return_sequences=False, batch_input_shape=(None, None, 39)))


es = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1E-4, verbose=2, patience=5)
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])
out = model.fit(xtrain, ytrain, batch_size=1, verbose=2, epochs=1000, callbacks=[es])


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).